In [1]:
# from google.transit import gtfs_realtime_pb2
import requests
import pandas as pd
import os

# import protobuf
import gtfs_realtime_pb2
import datetime
from time import strftime, localtime
from protobuf_to_dict import protobuf_to_dict
from datetime import datetime
import math
from datetime import datetime
import time


In [2]:
seen_trip_ids = set()
while True:
    feed = gtfs_realtime_pb2.FeedMessage()
    response = requests.get(
        "https://s3.amazonaws.com/kcm-alerts-realtime-prod/tripupdates.pb"
    )
    feed.ParseFromString(response.content)

    buses_dict = protobuf_to_dict(feed)
    df = pd.json_normalize(buses_dict["entity"])

    data_list = []

    for index, row in df.iterrows():
        stop_time_updates = row["trip_update.stop_time_update"]
        trip_id = row["trip_update.trip.trip_id"]
        start_date = row["trip_update.trip.start_date"]
        time_stamp = row["trip_update.timestamp"]
        # stop_id = row["trip_update.stop_time_update.stop_sequence"]

        date = datetime.strptime(start_date, "%Y%m%d")
        formatted_date = date.strftime("%B %d, %Y")
        date_day = datetime.strptime(formatted_date, "%B %d, %Y")

        date_cleaned = date.strftime("%Y-%m-%d")
        day_of_week = date_day.strftime("%A")

        for dictionary in stop_time_updates:
            stop_time_dict = dictionary.get("stop_sequence", {})
            arrival_dict = dictionary.get("arrival", {})
            delay = arrival_dict.get("delay", 0)
            stop = stop_time_dict
            stop_id = dictionary.get("stop_id", {})
            # stop_sequence = arrival_dict.get('stop_sequence', 0)

            if delay:
                if not math.isnan(float(time_stamp)):
                    datetime_obj = datetime.fromtimestamp(time_stamp)
                    formatted_datetime = datetime_obj.strftime("%H:%M:%S")

                data_list.append(
                    {
                        "trip_id": trip_id,
                        "start_date": formatted_date,
                        "delay": delay,
                        "day_of_week": day_of_week,
                        "time_stamp": formatted_datetime,
                        "stop_sequence": stop,
                        "stop_id": stop_id,
                    }
                )
                        # Add trip ID to the set of seen trip IDs
                seen_trip_ids.add(trip_id)

            new_df = pd.DataFrame(
                data_list,
                columns=[
                    "trip_id",
                    "start_date",
                    "delay",
                    "day_of_week",
                    "time_stamp",
                    "stop_sequence",
                    "stop_id",
                ],
            )

        if os.path.exists(
            "C:/Users/jcele/Desktop/arrival_time/analysis-/gathered_data/new_df"
        ):
            new_df.to_csv("gathered_data/new_df", mode="a", header=False, index=False)

        else:
            new_df.to_csv("gathered_data/new_df", index=False)

        new_df = pd.DataFrame()

    time.sleep(3600)